In [1]:
import pandas as pd
import json


In [2]:
df = pd.read_csv('demand_processed.csv')

In [3]:

column_mapping = {
    col: 'economic' if col.startswith('0 **Values Framework Pillar**') else
        'functional' if col.startswith('1 **Values Framework Pillar**') else
        'emotional' if col.startswith('2 **Values Framework Pillar**') else
        'social' if col.startswith('3 **Values Framework Pillar**') else
        'societal' if col.startswith('4 **Values Framework Pillar**') else
        'original_tweets' if col.startswith('wsai_content_0') else
        col
    for col in df.columns
}

# Rename the columns
df = df.rename(columns=column_mapping)

In [4]:
df['economic'][0]

'[\n  {"value": "Economic", "entity": "Open AI", "demand": 80},\n  {"value": "Economic", "entity": "Google", "demand": 75},\n  {"value": "Economic", "entity": "Microsoft", "demand": 85},\n  {"value": "Economic", "entity": "Meta", "demand": 70},\n  {"value": "Economic", "entity": "IBM", "demand": 65}\n]'

In [5]:
df.columns

Index(['original_author', 'original_tweets', 'Page_Rank', 'Rank', 'topic',
       'author_display_name', 'bio', 'country', 'num_followers',
       'Footprint_Influence_Score', 'economic', 'functional', 'emotional',
       'social', 'societal'],
      dtype='object')

In [6]:
# List of columns to clean
value_columns = ['economic', 'functional', 'emotional', 'social', 'societal']

# Clean newlines and perform additional cleanup
for col in value_columns:
    # Remove newlines
    df[col] = df[col].str.replace('\n', '')
    
    # Remove any extra whitespace
    df[col] = df[col].str.strip()
    
    # Remove empty strings and convert to NaN
    df[col] = df[col].replace('', pd.NA)
    
    # Drop rows where all value columns are NaN
    df = df.dropna(subset=value_columns, how='all')

In [7]:
len(df)

300

In [8]:
def preprocess_demand_data(df):
    """
    Preprocesses demand data by validating JSON structure and content.
    
    Args:
        df (pandas.DataFrame): Input dataframe with columns [economic, functional, emotional, social, societal]
    
    Returns:
        tuple: (cleaned_df, malformed_df) where:
            - cleaned_df contains only valid rows
            - malformed_df contains rejected rows with reason for rejection
    """
    
    # Create copy of original dataframe
    cleaned_df = df.copy()
    
    # Create list to store malformed rows and their reasons
    malformed_rows = []
    
    def validate_json_array(json_str, row_idx, column):
        try:
            # Parse JSON string
            data = json.loads(json_str)
            
            # Check if it's a list with exactly 5 entries
            if not isinstance(data, list) or len(data) != 5:
                return False, f"Invalid array length in {column}"
            
            # Expected companies
            expected_companies = {"Open AI", "Google", "Microsoft", "Meta", "IBM"}
            
            # Validate each entry
            companies_found = set()
            for entry in data:
                # Check if entry has all required fields
                if not all(key in entry for key in ["value", "entity", "demand"]):
                    return False, f"Missing required fields in {column}"
                
                # Check if demand is numeric
                if not isinstance(entry["demand"], (int, float)):
                    return False, f"Invalid demand value in {column}"
                
                # Check if entity is one of expected companies
                if entry["entity"] not in expected_companies:
                    return False, f"Invalid company name in {column}"
                
                # Add company to found set
                companies_found.add(entry["entity"])
            
            # Check if all companies are present
            if companies_found != expected_companies:
                return False, f"Missing companies in {column}"
            
            return True, "Valid"
            
        except json.JSONDecodeError:
            return False, f"Invalid JSON format in {column}"
        except Exception as e:
            return False, f"Unexpected error in {column}: {str(e)}"
    
    # Iterate through each row
    for idx, row in df.iterrows():
        row_valid = True
        error_reasons = []
        
        # Check each column
        for column in ["economic", "functional", "emotional", "social", "societal"]:
            is_valid, reason = validate_json_array(row[column], idx, column)
            if not is_valid:
                row_valid = False
                error_reasons.append(reason)
        
        # If row is not valid, add to malformed rows and mark for removal
        if not row_valid:
            malformed_rows.append({
                "row_index": idx,
                "reasons": "; ".join(error_reasons),
                **row
            })
            cleaned_df.drop(idx, inplace=True)
    
    # Create dataframe of malformed rows
    malformed_df = pd.DataFrame(malformed_rows)
    
    # Reset index of cleaned dataframe
    cleaned_df.reset_index(drop=True, inplace=True)
    
    print(f"Processed {len(df)} rows:")
    print(f"- {len(cleaned_df)} valid rows")
    print(f"- {len(malformed_df)} malformed rows")
    
    return cleaned_df, malformed_df

In [9]:
cleaned_df, malformed_df = preprocess_demand_data(df)

Processed 300 rows:
- 299 valid rows
- 1 malformed rows


In [11]:
malformed_df

,row_index,reasons,original_author,original_tweets,Page_Rank,Rank,topic,author_display_name,bio,country,num_followers,Footprint_Influence_Score,economic,functional,emotional,social,societal
0,4,Invalid JSON format in emotional,BrianRoemmele,Comparing the rather expensive OpenAI Sora to ...,0.010426,5,Open_Source,Brian Roemmele,we can only see what we think is possible...,United States,323377,8,"[ {""value"": ""Economic"", ""entity"": ""Open AI"", ...","[ {""value"": ""Functional"", ""entity"": ""Open AI""...","[ {""value"": ""Emotional"", ""entity"": ""Open AI"",...","[ {""value"": ""Social"", ""entity"": ""Open AI"", ""d...","[ {""value"": ""Societal"", ""entity"": ""Open AI"", ..."


In [ ]:
cleaned_df

In [20]:
# df.to_csv('demand_processed_staged.csv', index=False)

In [14]:
# Define the columns we want
metrics_columns = ['economic', 'functional', 'emotional', 'social', 'societal']
cleaned_df[metrics_columns].to_csv('demand_processed_staged.csv', index=False)